# Testing lab device drivers

Below are several examples of how to use or test the drivers in the labdevices folder.
Just scroll through and play around.

## General Tests
Use the pyvisa package connecting to all kinds of different devices via GPIB, SERIAL, USB, ETHERNET

In [2]:
import pyvisa

In [3]:
rm = pyvisa.ResourceManager()
resources = rm.list_resources()
resources

('ASRL/dev/ttyS4::INSTR', 'ASRL/dev/ttyUSB0::INSTR')

In [32]:
device = rm.open_resource(
    resources[1],
    timeout=100,
    encoding='ascii',
    parity=pyvisa.constants.Parity.none,
    baud_rate=9600,
    data_bits=8,
    stop_bits=pyvisa.constants.StopBits.one,
    flow_control=pyvisa.constants.VI_ASRL_FLOW_NONE,
    write_termination='\r\n',
    read_termination='\r\n'
)


In [70]:
device.query('AYT')

'TPG362,PTG28290,44998061,010300,010100'

In [87]:
device.query('\x05').split(',')

['0001']

In [62]:
device.write('\x03')

(3, <StatusCode.success: 0>)

In [88]:
device.close()

## Allied Vision

In [1]:
from labdevices import allied_vision

In [2]:
ccd = allied_vision.Manta(camera_id="DEV_000F314E6DE1")

Available cameras : ['DEV_000F314E6DE1']


In [3]:
ccd.initialize()

Connected to camera : DEV_000F314E6DE1


In [4]:
data = ccd.takeSingleImg()

In [5]:
data

array([[4, 1, 3, ..., 4, 2, 3],
       [4, 2, 1, ..., 1, 0, 1],
       [2, 0, 1, ..., 2, 1, 1],
       ...,
       [2, 0, 1, ..., 2, 2, 3],
       [0, 0, 1, ..., 1, 4, 0],
       [3, 1, 0, ..., 1, 0, 2]], dtype=uint8)

## Thorlabs devices
Like e.g. the temperature sensor

In [1]:
from labdevices import thorlabs

In [2]:
sensor = thorlabs.TSP01('USB0::4883::33016::M00416750::0::INSTR')

In [3]:
sensor.initialize()

Connected to Thorlabs,TSP01,M00416750,1.2.0.


In [5]:
sensor.close()

In [4]:
sensor.temperature_probe1()

21.38211

## Pfeiffer Vacuum devices
Like vacuum gauge controllers

In [1]:
from labdevices import pfeiffer_vacuum

In [2]:
sensor = pfeiffer_vacuum.TPG362()

In [3]:
sensor.initialize()

ValueError: 'parity' is not a valid attribute for type TCPIPSocket

In [4]:
sensor.idn()

{'Type': 'TPG362',
 'Model No.': 'PTG28290',
 'Serial No.': '44998061',
 'Firmware version': '010300',
 'Hardware version': '010100'}

In [5]:
sensor.error_status()

('0000', 'No error')

In [6]:
sensor.pressure_gauge(1)

(0.0, (5, 'No sensor (output: 5,2.0000E-2 [mbar])'))

In [7]:
sensor.pressure_gauges()

(0.0,
 (5, 'No sensor (output: 5,2.0000E-2 [mbar])'),
 0.0,
 (5, 'No sensor (output: 5,2.0000E-2 [mbar])'))

In [12]:
sensor.close()

In [7]:
sensor.pressure_unit()

'hPascal'

In [11]:
sensor.temperature()

23

### Trying to talk to Pfeiffer gauge controllers via Ethernet

In [2]:
import pyvisa

In [3]:
rm = pyvisa.ResourceManager()

In [4]:
rm.list_resources()

('ASRL/dev/ttyS4::INSTR',)

In [9]:
devi = rm.open_resource(
    'TCPIP::192.168.1.10::7776::SOCKET',
    #timeout=100,
    #encoding='ascii',
    #parity=pyvisa.constants.Parity.none,
    #baud_rate=9600,
    #data_bits=8,
    #stop_bits=pyvisa.constants.StopBits.one,
    #flow_control=pyvisa.constants.VI_ASRL_FLOW_NONE,
    write_termination='\r',
    #read_termination='\r\n' 
)

In [7]:
devi.close()

In [10]:
devi.query('MV')

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

## Applied Motion Products
For the control of e.g. the stepper motors of the rotary feedthroughs in the spectrometer chamber.

In [1]:
from labdevices import applied_motion_products

In [2]:
# Available IP addresses are 10.0.0.102 and 10.0.0.103. See also the IP table on the wiki.
device = applied_motion_products.STF03D('10.0.0.103')

In [3]:
device.initialize()

Connected to rotary feedthrough with IP=10.0.0.102.


In [23]:
device.close()

Closed rotary feedthrough with IP=10.0.0.102.


In [4]:
device.is_moving()

False

In [16]:
accel = ''
device.deceleration(accel)
device.acceleration(accel)

'AC=1'

In [19]:
device.speed()

'2'

In [12]:
device.idle_current(0.5)

'%'

In [12]:
device.move_relative(40)

Move by/to 40 degrees, equiv. to 21333 steps.


In [22]:
device.microstep()

3

## Rohde & Schwarz

In [1]:
from labdevices import rohde_schwarz
import numpy as np
device = rohde_schwarz.FPC1000('10.0.0.90')


In [2]:
device.initialize()

Connected to Rohde&Schwarz,FPC1000,1328.6660K02/102749,V1.50


In [7]:
device.close()

Connection to FPC1000 closed!


In [3]:
device.query('FORM:DATA?')

'ASC'

In [6]:
# Acquire the trace and save to file.
# Get trace sometimes times out. I don't know why... Any help is appreciated.
x, y = device.get_trace()
np.savetxt('test.txt', np.column_stack([x,y])) 

In [5]:
device.query('SYST:ERR:ALL?')

'-410,"Query interrupted",-410,"Query interrupted"'